In [ ]:
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

--2021-01-29 16:06:17--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  38.5MB/s    in 7.5s    

2021-01-29 16:06:25 (23.0 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2021-01-29 16:06:25--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  16.5MB/s    in 4.3s    

2021-01-29 16:06:30 (14.2 MB/s) - ‘test_32x32.mat’ saved [64275384/64

In [ ]:
import keras
from keras.layers.normalization import BatchNormalization
import numpy as np
import scipy.io as io
from keras.layers import Activation, Flatten,Softmax
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.models import Sequential
from keras import optimizers
from keras.backend import function
import pandas as pd
import tensorflow as tf


In [ ]:
train_set = io.loadmat("train_32x32.mat")
test_set = io.loadmat("test_32x32.mat")
x_train = train_set['X']
y_train = train_set['y']
x_test = test_set['X']
y_test = test_set['y']
# Reshaping test and training data sets
x_train = np.moveaxis(x_train,[3,0,1,2],[0,1,2,3])
x_train = x_train.astype('float32')
x_test = np.moveaxis(x_test,[3,0,1,2],[0,1,2,3])
x_test = np.asarray(x_test)
x_test = x_test.astype('float32')
#Normalisation des données
x_train /= x_train.max()
x_test /= x_test.max()
# Passage de classe comme integer à classe comme vecteur de dimension classes
y_test_RN = [y-1 for y in y_test]# la classe 1 corresponds au array qui à 1 en indice 0
y_train_RN = [y-1 for y in y_train]
classes = 10
y_train_RN = keras.utils.to_categorical(y_train_RN, classes)
y_test_RN = keras.utils.to_categorical(y_test_RN, classes)
 #Définition du type de data pour optimiser la mémoire
y_train_RN = y_train_RN.astype('int32')
y_test_RN = y_test_RN.astype('int32')
# Featurewise normalization
datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=False,
    zca_whitening=False, 
    zca_epsilon=1e-06)

datagen.fit(x_train)

In [ ]:
y_test_KNN=[k.item() for k in y_test]
y_train_KNN=[k.item() for k in y_train]

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
an_array=[1,2,14,6,0,9]
max_index_col = np.argmax(an_array, axis=0)
max_index_col

2

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Projet 3A')

In [ ]:
def deepnet2():
  model=Sequential()
  model.add(Conv2D(64, kernel_size=(3, 3),padding = 'same', activation='relu', input_shape=x_train.shape[1:]))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),  padding = 'valid', name='S1'))
  model.add(Conv2D(128, kernel_size=(3, 3), padding = 'valid', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),  padding = 'same', name='S2'))
  model.add(Conv2D(128, kernel_size=(3, 3),  strides = (1,1), padding = 'valid',  activation='relu'))
  model.add(Conv2D(128, kernel_size=(3, 3),  strides = (1,1), padding = 'valid',  activation='relu'))
  model.add(Conv2D(128, kernel_size=(3, 3),  strides = (1,1), padding = 'valid',  activation='relu'))
  model.add(Dropout(0.5)) 
  model.add(Flatten())
  model.add(Dense(classes, activation='softmax'))
  adam = optimizers.Adam(lr = 0.001)
  model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer= adam,
                  metrics=['accuracy'])
  return model

In [ ]:
model=deepnet2()

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
S1 (MaxPooling2D)            (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 128)       73856     
_________________________________________________________________
S2 (MaxPooling2D)            (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 5, 5, 128)         147584    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 3, 3, 128)         147584    
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 1, 1, 128)        

In [ ]:
scores=[]
x_train_10000=x_train[:10000]

In [ ]:
def takeSecond(elem):
    return elem[1]

In [ ]:
def somme_array(L):
  somme=np.zeros(len(L[0]))
  for l in L:
    somme=somme+l
  return somme

In [ ]:
model.predict(x_train[0:1])[0]

array([9.9862885e-01, 1.3458154e-04, 2.0203834e-04, 3.1555185e-04,
       8.8591933e-06, 5.3440385e-06, 6.0649344e-04, 3.1276793e-06,
       7.7708155e-06, 8.7479806e-05], dtype=float32)

In [ ]:
def neighbors(x,d,k):
  distance=[]
  voisin=[]
  for i in range(len(d)):
    L=[i]
    L.append(np.linalg.norm(x-d[i]))
    distance.append(L)
  distance=sorted(distance,key=takeSecond)
  for j in range(k):
    voisin.append(distance[j][0])
  return voisin



In [ ]:
def predict(x,d,k,model):
  voisins=neighbors(x,d,k)
  neig=[]
  for N in voisins:
    neig.append(model.predict(x_train[N:N+1])[0])
  v=somme_array(neig)
  classe=np.argmax(v, axis=0)+1
  return classe 

In [ ]:
def predict_data(x_t,x_tr,k,model):
  prediction=[]
  for k in range(len(x_t)):
    prediction.append(predict(x_t[k],x_tr,k,model))
  return prediction

#Conv2D 50

In [ ]:
get_conv50_output = function([model.layers[0].input],
                                  [model.layers[6].output])
layer_output_conv50 = get_conv50_output([x_train_10000])

In [ ]:
conv50=np.array(layer_output_conv50[0]).reshape(10000,128)

In [ ]:
layer_output_conv50_test = get_conv50_output([x_test[:5000]])

In [ ]:
conv50_test=np.array(layer_output_conv50_test[0]).reshape(5000,128)

In [ ]:
prediction=[]
for k in range(len(conv50_test)):
  prediction.append(predict(conv50_test[k],conv50,5,model))
prediction

[5,
 2,
 1,
 10,
 6,
 1,
 9,
 1,
 1,
 8,
 3,
 6,
 5,
 1,
 4,
 4,
 1,
 6,
 3,
 4,
 2,
 10,
 1,
 3,
 2,
 5,
 4,
 1,
 4,
 2,
 9,
 3,
 8,
 6,
 10,
 1,
 5,
 1,
 1,
 2,
 8,
 1,
 6,
 9,
 9,
 8,
 1,
 2,
 10,
 6,
 9,
 1,
 5,
 1,
 9,
 2,
 4,
 5,
 1,
 9,
 2,
 9,
 2,
 4,
 1,
 10,
 6,
 9,
 3,
 2,
 3,
 5,
 1,
 1,
 4,
 6,
 2,
 6,
 3,
 1,
 4,
 1,
 7,
 1,
 2,
 1,
 7,
 1,
 10,
 1,
 4,
 4,
 6,
 4,
 4,
 2,
 7,
 1,
 7,
 7,
 4,
 8,
 2,
 3,
 2,
 2,
 3,
 1,
 8,
 6,
 3,
 7,
 6,
 5,
 10,
 7,
 2,
 7,
 4,
 2,
 8,
 8,
 1,
 7,
 4,
 9,
 1,
 4,
 2,
 9,
 6,
 4,
 8,
 10,
 8,
 2,
 10,
 1,
 10,
 3,
 1,
 2,
 4,
 6,
 7,
 2,
 1,
 3,
 5,
 3,
 1,
 5,
 9,
 1,
 8,
 1,
 3,
 6,
 4,
 6,
 4,
 9,
 3,
 6,
 1,
 10,
 1,
 2,
 6,
 2,
 7,
 5,
 2,
 2,
 1,
 4,
 2,
 9,
 6,
 7,
 1,
 10,
 2,
 1,
 4,
 8,
 3,
 2,
 1,
 6,
 2,
 6,
 1,
 7,
 2,
 3,
 6,
 1,
 1,
 3,
 7,
 2,
 3,
 5,
 10,
 2,
 1,
 10,
 4,
 7,
 1,
 10,
 2,
 3,
 7,
 3,
 1,
 1,
 1,
 2,
 10,
 4,
 7,
 1,
 2,
 3,
 6,
 2,
 2,
 9,
 2,
 5,
 7,
 4,
 1,
 1,
 3,
 4,
 3,
 2,
 7,
 1,
 1,
 4,
 1,
 7,


In [ ]:
right=0
for k in range(5000):
  if prediction[k]==y_test_KNN[k]:
    right=right+1
right


4590

In [ ]:
score=right/5000
scores.append(score)


In [ ]:
scores

[0.918, 0.918]

# Conv2D 40


In [ ]:
get_conv40_output = function([model.layers[0].input],
                                  [model.layers[5].output])
layer_output_conv40 = get_conv40_output([x_train_10000])

In [ ]:
conv40=np.array(layer_output_conv40[0]).reshape(10000,1152)

In [ ]:
layer_output_conv40_test = get_conv40_output([x_test[:5000]])

In [ ]:
conv40_test=np.array(layer_output_conv40_test[0]).reshape(5000,1152)

In [ ]:
prediction=[]
for k in range(len(conv40_test)):
  prediction.append(predict(conv40_test[k],conv40,5,model))
prediction

[5,
 2,
 1,
 10,
 6,
 1,
 9,
 1,
 1,
 8,
 3,
 8,
 5,
 1,
 4,
 4,
 1,
 6,
 3,
 4,
 2,
 10,
 1,
 9,
 2,
 5,
 4,
 1,
 4,
 2,
 9,
 3,
 5,
 6,
 10,
 1,
 5,
 1,
 1,
 2,
 9,
 1,
 6,
 9,
 9,
 4,
 1,
 2,
 10,
 6,
 9,
 1,
 5,
 1,
 9,
 2,
 4,
 5,
 1,
 5,
 2,
 9,
 2,
 4,
 1,
 1,
 6,
 9,
 3,
 2,
 3,
 5,
 1,
 1,
 4,
 6,
 2,
 6,
 3,
 1,
 4,
 1,
 7,
 1,
 2,
 1,
 7,
 1,
 10,
 1,
 4,
 4,
 6,
 4,
 4,
 2,
 7,
 1,
 7,
 7,
 4,
 8,
 2,
 5,
 2,
 2,
 3,
 1,
 8,
 6,
 3,
 7,
 6,
 3,
 10,
 7,
 2,
 7,
 4,
 2,
 8,
 8,
 1,
 7,
 4,
 10,
 1,
 4,
 2,
 9,
 6,
 1,
 6,
 9,
 8,
 2,
 10,
 1,
 10,
 3,
 1,
 2,
 4,
 6,
 7,
 2,
 1,
 3,
 5,
 3,
 1,
 5,
 9,
 1,
 8,
 1,
 8,
 6,
 4,
 6,
 4,
 9,
 3,
 6,
 1,
 10,
 1,
 2,
 5,
 2,
 7,
 8,
 2,
 2,
 1,
 4,
 2,
 9,
 6,
 7,
 1,
 10,
 2,
 1,
 4,
 8,
 3,
 2,
 1,
 6,
 2,
 6,
 1,
 7,
 2,
 3,
 6,
 1,
 1,
 3,
 7,
 2,
 1,
 5,
 10,
 2,
 1,
 10,
 4,
 7,
 1,
 10,
 2,
 3,
 7,
 3,
 1,
 1,
 1,
 2,
 10,
 1,
 7,
 7,
 2,
 1,
 6,
 2,
 2,
 9,
 2,
 5,
 7,
 4,
 1,
 1,
 3,
 4,
 3,
 3,
 7,
 1,
 1,
 4,
 1,
 7,
 

In [ ]:
right=0
for k in range(5000):
  if prediction[k]==y_test_KNN[k]:
    right=right+1
right

4432

In [ ]:
score=right/5000
scores.append(score)

In [ ]:
scores

[0.918, 0.918, 0.8864]

#Conv2D 30

In [ ]:
get_conv30_output = function([model.layers[0].input],
                                  [model.layers[4].output])
layer_output_conv30 = get_conv30_output([x_train_10000])

In [ ]:
conv30=np.array(layer_output_conv30[0]).reshape(10000,3200)

In [ ]:
layer_output_conv30_test = get_conv30_output([x_test[:5000]])

In [ ]:
conv30_test=np.array(layer_output_conv30_test[0]).reshape(5000,3200)

In [ ]:
prediction=[]
for k in range(len(conv30_test)):
  prediction.append(predict(conv30_test[k],conv30,5,model))
prediction

[5,
 2,
 1,
 10,
 6,
 1,
 9,
 1,
 1,
 8,
 3,
 6,
 5,
 1,
 4,
 4,
 1,
 10,
 3,
 4,
 2,
 10,
 1,
 3,
 2,
 5,
 4,
 1,
 4,
 2,
 10,
 3,
 6,
 6,
 10,
 1,
 5,
 1,
 1,
 2,
 2,
 1,
 6,
 5,
 2,
 6,
 1,
 2,
 10,
 6,
 9,
 1,
 5,
 1,
 9,
 6,
 4,
 5,
 7,
 5,
 2,
 2,
 2,
 4,
 1,
 9,
 6,
 9,
 3,
 2,
 3,
 5,
 1,
 1,
 4,
 6,
 2,
 5,
 3,
 1,
 4,
 1,
 7,
 1,
 2,
 1,
 7,
 1,
 10,
 1,
 4,
 4,
 6,
 4,
 4,
 2,
 7,
 1,
 7,
 7,
 4,
 8,
 2,
 5,
 2,
 2,
 3,
 1,
 5,
 6,
 3,
 7,
 5,
 10,
 9,
 7,
 2,
 7,
 10,
 2,
 8,
 8,
 1,
 1,
 1,
 10,
 3,
 4,
 2,
 2,
 6,
 8,
 6,
 9,
 8,
 2,
 9,
 1,
 10,
 3,
 2,
 2,
 4,
 5,
 7,
 2,
 1,
 3,
 5,
 3,
 1,
 5,
 9,
 1,
 8,
 1,
 10,
 6,
 1,
 6,
 4,
 9,
 3,
 6,
 1,
 6,
 1,
 2,
 6,
 2,
 7,
 2,
 2,
 2,
 1,
 4,
 2,
 9,
 6,
 7,
 1,
 10,
 2,
 1,
 4,
 8,
 3,
 2,
 1,
 6,
 2,
 6,
 1,
 7,
 7,
 3,
 6,
 1,
 1,
 3,
 7,
 2,
 3,
 5,
 10,
 2,
 1,
 10,
 4,
 7,
 1,
 10,
 2,
 3,
 7,
 3,
 1,
 1,
 1,
 2,
 10,
 2,
 7,
 7,
 2,
 9,
 1,
 2,
 2,
 9,
 2,
 5,
 7,
 4,
 1,
 7,
 7,
 6,
 3,
 2,
 7,
 1,
 1,
 4,
 1,
 7,

In [ ]:
right=0
for k in range(5000):
  if prediction[k]==y_test_KNN[k]:
    right=right+1
right

4111

In [ ]:
score=right/5000
scores.append(score)

In [ ]:
scores

[0.918, 0.918, 0.8864, 0.8222]

# Maxpooling S2

In [ ]:
get_S2_output = function([model.layers[0].input],
                                  [model.layers[3].output])
layer_output_S2 = get_S2_output([x_train_10000])

In [ ]:
S2=np.array(layer_output_S2[0]).reshape(10000,6272)

In [ ]:
layer_output_S2_test = get_S2_output([x_test[:5000]])

In [ ]:
S2_test=np.array(layer_output_S2_test[0]).reshape(5000,6272)

In [ ]:
prediction=[]
for k in range(len(S2_test)):
  prediction.append(predict(S2_test[k],S2,5,model))
prediction

[3,
 2,
 1,
 10,
 4,
 1,
 2,
 1,
 1,
 8,
 3,
 5,
 5,
 1,
 4,
 4,
 1,
 10,
 3,
 4,
 2,
 10,
 1,
 1,
 2,
 5,
 4,
 1,
 4,
 2,
 10,
 3,
 4,
 10,
 10,
 1,
 5,
 1,
 1,
 2,
 3,
 1,
 6,
 5,
 2,
 4,
 1,
 2,
 10,
 6,
 9,
 1,
 5,
 1,
 9,
 2,
 5,
 5,
 1,
 3,
 2,
 4,
 2,
 1,
 1,
 1,
 4,
 9,
 3,
 2,
 9,
 5,
 1,
 1,
 1,
 6,
 2,
 5,
 3,
 1,
 1,
 1,
 7,
 1,
 2,
 1,
 7,
 1,
 10,
 1,
 6,
 4,
 6,
 4,
 4,
 2,
 7,
 1,
 7,
 1,
 4,
 8,
 2,
 5,
 6,
 2,
 3,
 1,
 4,
 6,
 3,
 1,
 5,
 2,
 8,
 7,
 2,
 7,
 10,
 2,
 10,
 8,
 1,
 1,
 1,
 10,
 2,
 4,
 2,
 2,
 6,
 1,
 6,
 4,
 8,
 2,
 2,
 1,
 10,
 2,
 4,
 2,
 4,
 5,
 7,
 2,
 1,
 4,
 5,
 3,
 1,
 5,
 9,
 1,
 8,
 1,
 10,
 5,
 1,
 1,
 4,
 9,
 2,
 6,
 1,
 10,
 1,
 2,
 1,
 2,
 7,
 2,
 2,
 2,
 1,
 4,
 6,
 9,
 6,
 7,
 1,
 10,
 2,
 1,
 4,
 8,
 3,
 2,
 1,
 8,
 2,
 5,
 1,
 1,
 2,
 3,
 5,
 1,
 1,
 3,
 7,
 2,
 2,
 5,
 10,
 2,
 4,
 10,
 4,
 7,
 1,
 10,
 2,
 3,
 7,
 3,
 1,
 1,
 1,
 2,
 10,
 2,
 7,
 7,
 2,
 10,
 1,
 2,
 2,
 9,
 2,
 5,
 7,
 4,
 4,
 1,
 2,
 1,
 3,
 1,
 7,
 1,
 1,
 4,
 1,


In [ ]:
right=0
for k in range(5000):
  if prediction[k]==y_test_KNN[k]:
    right=right+1
right

3521

In [ ]:
score=right/5000
scores.append(score)
scores

[0.918, 0.918, 0.8864, 0.8222, 0.7042]

#Conv2D 20

In [ ]:
get_conv20_output = function([model.layers[0].input],
                                  [model.layers[2].output])
layer_output_conv20 = get_conv20_output([x_train_10000])

In [ ]:
conv20=np.array(layer_output_conv20[0]).reshape(10000,25088)

In [ ]:
layer_output_conv20_test = get_conv20_output([x_test[:5000]])

In [ ]:
conv20_test=np.array(layer_output_conv20_test[0]).reshape(5000,25088)

In [ ]:
prediction=[]
for k in range(len(conv20_test)):
  prediction.append(predict(conv20_test[k],conv20,5,model))
prediction

[5,
 2,
 1,
 10,
 4,
 1,
 2,
 1,
 1,
 3,
 3,
 1,
 5,
 1,
 4,
 4,
 1,
 10,
 3,
 4,
 2,
 10,
 1,
 3,
 2,
 5,
 4,
 1,
 4,
 2,
 10,
 3,
 4,
 5,
 10,
 1,
 3,
 1,
 1,
 2,
 2,
 1,
 6,
 1,
 1,
 4,
 1,
 2,
 10,
 6,
 9,
 1,
 5,
 1,
 10,
 2,
 4,
 5,
 1,
 5,
 2,
 10,
 2,
 1,
 1,
 1,
 4,
 5,
 5,
 2,
 9,
 5,
 1,
 1,
 1,
 6,
 2,
 5,
 3,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 7,
 1,
 10,
 1,
 4,
 4,
 6,
 4,
 1,
 2,
 2,
 1,
 7,
 1,
 4,
 6,
 2,
 5,
 6,
 2,
 3,
 1,
 4,
 6,
 1,
 1,
 1,
 2,
 8,
 7,
 2,
 1,
 9,
 2,
 10,
 9,
 1,
 1,
 1,
 1,
 1,
 4,
 2,
 2,
 5,
 5,
 1,
 2,
 8,
 2,
 2,
 1,
 10,
 3,
 1,
 2,
 4,
 5,
 7,
 2,
 1,
 4,
 5,
 3,
 1,
 5,
 9,
 1,
 4,
 10,
 4,
 5,
 1,
 6,
 4,
 9,
 9,
 6,
 1,
 9,
 1,
 2,
 3,
 2,
 7,
 2,
 2,
 2,
 1,
 4,
 6,
 9,
 6,
 7,
 1,
 10,
 1,
 1,
 4,
 8,
 3,
 2,
 1,
 10,
 2,
 5,
 1,
 1,
 2,
 3,
 5,
 1,
 1,
 3,
 7,
 2,
 2,
 5,
 6,
 2,
 1,
 10,
 4,
 7,
 1,
 10,
 2,
 3,
 7,
 3,
 1,
 1,
 1,
 2,
 10,
 2,
 7,
 7,
 1,
 1,
 1,
 2,
 2,
 9,
 2,
 5,
 1,
 4,
 4,
 1,
 7,
 1,
 3,
 1,
 7,
 1,
 10,
 4,
 1,
 7

In [ ]:
right=0
for k in range(5000):
  if prediction[k]==y_test_KNN[k]:
    right=right+1
right

3363

In [ ]:
score=right/5000
scores.append(score)
scores

[0.918, 0.918, 0.8864, 0.8222, 0.7042, 0.6726]